<a href="https://colab.research.google.com/github/cauancalmeida/Data_Science_studies/blob/main/Exercicio_amostragem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Carregando base de dados

In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
dataset = pd.read_csv('credit_data.csv')

In [4]:
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


##Amostragem aleatória simples

In [5]:
def amostragem_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras, random_state = 1)

##Amostragem sistemática

In [7]:
def amostragem_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras
  random.seed(1)
  inicio = random.randint(0, intervalo)
  indices = np.arange(inicio, len(dataset), step = intervalo)
  return dataset.iloc[indices]

##Amostragem por grupos

In [9]:
def amostragem_agrupada(dataset, numero_grupos):
    intervalo = len(dataset) / numero_grupos
    grupos = []
    id_grupo = 0
    contagem = 0
    for index, row in dataset.iterrows():
        grupos.append(id_grupo)
        contagem += 1
        if contagem > intervalo:
            contagem = 0
            id_grupo += 1
    dataset['grupo'] = grupos
    random.seed(1)
    grupo_selecionado = random.randint(0, numero_grupos)
    return dataset[dataset['grupo'] == grupo_selecionado]

##Amostra estratificada

In [30]:
from typing import DefaultDict
def amostragem_estratificada(dataset):
  split = StratifiedShuffleSplit(test_size=None ,random_state=1)
  for _, y in split.split(dataset, dataset['c#default']):
    df_y = dataset.iloc[y]
  return df_y

##Amostragem Reservatorio

In [33]:
def amostragem_reervatorio(dataset, amostras):
  random.seed(1)
  stream = []
  for i in range(len(dataset)):#lista dos IDs dos stream vindos do dataset
    stream.append(i)
  i = 0
  reservatorio = [0] * amostras #Reserva os IDs dos registros selecionados
  for i in range(amostras):#inicia o reservatorio
    reservatorio[i] = stream[i]
  while i< len(dataset):
    j = random.randrange(i+1)
    if j < amostras:
      reservatorio[j] = stream[i] #reservatorio recebe os valores sorteados
    i += 1
  return dataset.iloc[reservatorio]

##Comparativo dos resultados

In [43]:
def comparativo_resultados(dataset, amostras, coluna):
  resultados = {}
  resultados['Aleatória Simples'] = amostragem_aleatoria_simples(dataset, amostras)[coluna].mean()
  resultados['Sistemática'] = amostragem_sistematica(dataset, amostras)[coluna].mean()
  resultados['Agrupamento'] = amostragem_agrupada(dataset, amostras)[coluna].mean()
  resultados['Estratificada'] = amostragem_estratificada(dataset)[coluna].mean()
  resultados['Reservatório'] = amostragem_reervatorio(dataset, amostras)[coluna].mean()
  return resultados

In [50]:
media_age = dataset['age'].mean()
media_income=dataset['income'].mean()
media_loan= dataset['loan'].mean()